In [2]:
# import stuff
import numpy as np
import time

from datasets import load_dataset
import dspy
import weaviate

In [3]:
# dspy setup
lm = dspy.LM(
    "openai/gpt-4.1",
    cache=False,
    api_key=os.environ["OPENAI_API_KEY"]
)

dspy.configure(lm=lm, track_usage=True)

lm("say hello")

['Hello! How can I help you today? 😊']

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [input_value=Message(content='Hello! H...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


In [7]:
# oracle context test
from typing import Any, Literal
from weaviate.classes.query import Filter

def fetch_object(
    collection: Any, 
    pdf_id: str, 
    return_image_or_text: Literal["image", "text"]
) -> str | dspy.Image:
    if return_image_or_text == "image":
        response = collection.query.fetch_objects(
            filters=Filter.by_property("dataset_id").like(pdf_id),
            return_properties=["base64_str"]
        )
        return response.objects[0].properties["base64_str"]
    elif return_image_or_text == "text":
        response = collection.query.fetch_objects(
            filters=Filter.by_property("dataset_id").like(pdf_id),
            return_properties=["content"]
        )
        return response.objects[0].properties["content"]
weaviate_client = weaviate.connect_to_weaviate_cloud(
    cluster_url=os.environ["WEAVIATE_URL"],
    auth_credentials=weaviate.auth.AuthApiKey(os.environ["WEAVIATE_API_KEY"])
)

collection = weaviate_client.collections.get("IRPapersText_Default")

fetch_object(collection, "8_1", "text")

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/weaviate/warnings.py:302: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(
/var/folders/41/8dp_379x15d8zz4ppsjthdw40000gn/T/ipykernel_96876/3365750985.py:27: ResourceWarning: unclosed <ssl.SSLSocket fd=318, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('10.0.0.233', 60566), raddr=('35.201.124.182', 443)>
  collection = weaviate_client.collections.get("IRPapersText_Default")


'Query Rewriting for Retrieval-Augmented Large Language Models\n\nXinbei Ma¹,², *, Yeyun Gong³, #, †, Pengcheng He⁴, #, Hai Zhao¹,², †, Nan Duan³\n¹Department of Computer Science and Engineering, Shanghai Jiao Tong University  \n²Key Laboratory of Shanghai Education Commission for Intelligent Interaction and Cognitive Engineering, Shanghai Jiao Tong University  \n³Microsoft Research Asia  \n⁴Microsoft Azure AI  \nsjtu_maxb@sjtu.edu.cn, zhaohai@cs.sjtu.edu.cn,  \n{yegong, nanduan}@microsoft.com, Herbert.he@gmail.com\n\n*Work done during an internship at ³Microsoft Research Asia. # Equal contribution. † Corresponding author.  \nThis paper was partially supported by Joint Research Project of Yangtze River Delta Science and Technology Innovation Community (No. 2022CSCGI1400).  \n¹https://github.com/xmbx/RAG-query-rewriting\n\nAbstract\n\nLarge Language Models (LLMs) play powerful, black-box readers in the retrieve-then-read pipeline, making remarkable progress in knowledge-intensive tasks.

In [6]:
class GenerateAnswerFromParameters(dspy.Signature):
    """Answer the question as well as you can."""

    question: str = dspy.InputField(description="The question to answer.")
    answer: str = dspy.OutputField(description="The answer to the question.")

qa_system = dspy.Predict(GenerateAnswerFromParameters)

qa_system(question="What is HyDE?")

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [input_value=Message(content='[[ ## an...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


Prediction(
    answer='HyDE can refer to several things depending on the context, but most commonly:\n\n1. **HyDE (Hybrid Density Estimator)**: In artificial intelligence and large language models (LLMs), HyDE stands for "Hypothetical Document Embeddings". It is a method, introduced by OpenAI, for enhancing retrieval-augmented generation (RAG) systems. HyDE works by generating a hypothetical answer to a user\'s query, then using the embedding (vector representation) of that hypothetical document to better match relevant real documents during retrieval from a database or corpus. This leads to improved information retrieval performance, as the search is guided by a synthesized, context-rich sample.\n\n2. **Hyde (Other meanings)**:\n   - It could refer to the character "Hyde" from the classic novel "Strange Case of Dr Jekyll and Mr Hyde" by Robert Louis Stevenson.\n   - In music, Hyde is a well-known Japanese singer.\n   - There are cities named Hyde (e.g., Hyde, Greater Manchester in En

In [ ]:
# define RAG systems

class GenerateAnswer(dspy.Signature):
    """Assess the context and answer the question."""

    question: str = dspy.InputField(description="The question to answer.")
    context: list[str] | list[dspy.Image] = dspy.InputField(description="The context to use to answer the question.")
    answer: str = dspy.OutputField(description="The answer to the question.")

class RAGSystem(dspy.Module):
    def __init__(self, collection: Any, images_or_text: Literal["image", "text"], k: int = 5):
        self.generate_answer = dspy.Predict(GenerateAnswer)
        self.collection = collection
        self.images_or_text = images_or_text
        self.k = k
    def _get_objects(self, question: str) -> list[str] | list[dspy.Image]:
        if self.images_or_text == "image":
            response = self.collection.query.hybrid(
                query=question,
                return_properties=["base64_str"],
                limit=self.k
            )
            objects = []
            for o in response.objects:
                objects.append(o.properties["base64_str"])
            # do PIL magic and convert these to dspy.Image objects
            return objects
        elif self.images_or_text == "text":
            response = self.collection.query.hybrid(
                query=question,
                return_properties=["content"],
                limit=self.k
            )
            objects = []
            for o in response.objects:
                objects.append(o.properties["content"])
            return objects
        
    def _fetch_oracle_context(
        oracle_context_id: str, 
    ) -> str | dspy.Image:
        if self.images_or_text == "image":
            response = collection.query.fetch_objects(
                filters=Filter.by_property("dataset_id").like(oracle_context_id),
                return_properties=["base64_str"]
            )
            return response.objects[0].properties["base64_str"]
        elif self.images_or_text == "text":
            response = collection.query.fetch_objects(
                filters=Filter.by_property("dataset_id").like(oracle_context_id),
                return_properties=["content"]
            )
            return response.objects[0].properties["content"]

    def __call__(
        self, 
        question: str, 
        context: list[str] | list[dspy.Image],
        oracle_context_id: str = None
    ) -> str:
        if oracle_context_id is None:
            context = self._get_objects(question)
        else:
            context = self._fetch_oracle_context(oracle_context_id)
        return self.generate_answer(question=question, context=context)

In [12]:
# llm as judge
class AssessAlignmentScore(dspy.Signature):
    """You are an expert grader assessing if a system's answer is semantically aligned with the correct answer.
    Only return True if the system answer has essentially the same meaning as the correct answer.
    If the system answer misses key aspects or meaning, return False.
    """

    question: str = dspy.InputField(description="The question asked.")
    system_answer: str = dspy.InputField(description="The answer generated by the system.")
    correct_answer: str = dspy.InputField(description="The reference answer containing the correct and complete information.")
    score: bool = dspy.OutputField(description="True if system_answer is equivalent in meaning to correct_answer, otherwise False.")

judge = dspy.Predict(AssessAlignmentScore)

test_question = "What is HyDE?"
correct_answer = "HyDE stands for Hypothetical Document Embeddings, a technique for improving retrieval in AI systems by generating hypothetical answers and using their embeddings."

# System answer missing key aspect (embeddings)
incorrect_answer = "HyDE is a technique for improving retrieval in AI systems by generating hypothetical answers."
# System answer rewords but covers all key ideas
acceptable_answer = "Hypothetical Document Embeddings (HyDE) is a method to help AI retrieval by creating hypothetical documents as sample answers and using their vector representations."

response = judge(question=test_question, system_answer=incorrect_answer, correct_answer=correct_answer)
print(response)
response = judge(question=test_question, system_answer=acceptable_answer, correct_answer=correct_answer)
print(response)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [input_value=Message(content='[[ ## sc...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


Prediction(
    score=False
)
Prediction(
    score=True
)


In [13]:
response.get_lm_usage()

{'openai/gpt-4.1': {'completion_tokens': 12,
  'prompt_tokens': 349,
  'total_tokens': 361,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0,
   'text_tokens': None},
  'prompt_tokens_details': {'audio_tokens': 0,
   'cached_tokens': 0,
   'text_tokens': None,
   'image_tokens': None}}}

In [ ]:
# load data
from datasets import load_dataset

queries = load_dataset("weaviate/irpapers-queries")["train"]

In [23]:
alignment_scores, input_tokens, output_tokens = [], [], []

K = 3

start = time.time()
for idx, query in enumerate(queries):
    if idx % 5 == 4:
        print(f"Processed {idx+1} queries in {time.time() - start} seconds...")
    test_query, ground_truth_answer = query["question"], query["answer"]
    qa_system_response = qa_system(
        question=test_query
    )
    usage_dict = qa_system_response.get_lm_usage()["openai/gpt-4.1"]
    input_tokens.append(usage_dict["prompt_tokens"])
    output_tokens.append(usage_dict["completion_tokens"])

    ensemble_votes = 0
    for judge_predictions in range(K):
        lm_judge_response = judge(
            question=test_query,
            system_answer=qa_system_response.answer,
            correct_answer=ground_truth_answer
        )
        if lm_judge_response.score:
            ensemble_votes += 1
    if ensemble_votes >= K / 2:
        alignment_scores.append(1)
    else:
        alignment_scores.append(0)

alignment_scores = np.array(alignment_scores)
input_tokens = np.array(input_tokens)
output_tokens = np.array(output_tokens)

print(alignment_scores.mean())
print(input_tokens.mean())
print(output_tokens.mean())


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [input_value=Message(content='[[ ## an...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [input_value=Message(content='[[ ## sc...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpec

Processed 5 queries in 16.03865694999695 seconds...


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [input_value=Message(content="[[ ## an...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


Processed 10 queries in 42.11020588874817 seconds...
Processed 15 queries in 61.140347957611084 seconds...
Processed 20 queries in 78.02888798713684 seconds...
Processed 25 queries in 112.45030188560486 seconds...
Processed 30 queries in 138.47028803825378 seconds...
Processed 35 queries in 178.7451891899109 seconds...
Processed 40 queries in 199.21194005012512 seconds...
Processed 45 queries in 230.06932997703552 seconds...
Processed 50 queries in 270.6622290611267 seconds...


KeyboardInterrupt: 

In [24]:
alignment_scores = np.array(alignment_scores)
input_tokens = np.array(input_tokens)
output_tokens = np.array(output_tokens)

print(alignment_scores.mean())
print(input_tokens.mean())
print(output_tokens.mean())

0.17647058823529413
170.28846153846155
123.32692307692308
